In [1]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 706.9 kB/s eta 0:00:00a 0:00:01


In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


/var/folders/24/4xzytrxn76jb3gxp4dplrtd40000gp/T/ipykernel_51994/4206453499.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0


11490434/11490434 [==============================] - 1s 0us/step


In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [5]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # number of model configurations to test
    executions_per_trial=3,  # number of times to train each model
    directory='my_dir',
    project_name='mnist_tuning')

tuner.search_space_summary()

tuner.search(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))


Trial 5 Complete [00h 01m 16s]
val_accuracy: 0.9811000029246012

Best val_accuracy So Far: 0.9811000029246012
Total elapsed time: 00h 06m 17s
INFO:tensorflow:Oracle triggered exit


In [6]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.

